In [1]:
import pandas as pd
import numpy as np
import warnings
import rpy2.robjects as ro
import rpy2.robjects.packages as rpackages
from rpy2.robjects import pandas2ri, StrVector
from rpy2.robjects.conversion import localconverter
import pandas as pd
from scipy.stats import pearsonr
from itertools import combinations
import scipy.stats
import scipy


confidence_level = 0.95

### The non Central F function
def Non_Central_CI_F(F_Statistic, df1, df2, confidence_level):
    Upper_Limit = 1 - (1 - confidence_level) / 2
    Lower_Limit = 1 - Upper_Limit
    Lower_CI_Difference_Value = 1

    def Lower_CI(F_Statistic, df1, df2, Upper_Limit, Lower_CI_Difference_Value):
        Lower_Bound = [0.001, F_Statistic / 2, F_Statistic]       
        while scipy.stats.ncf.cdf(F_Statistic, df1, df2, Lower_Bound[0]) < Upper_Limit:
            return [0, scipy.stats.f.cdf(F_Statistic, df1, df2)] if scipy.stats.f.cdf(F_Statistic, df1, df2) < Upper_Limit else None
            Lower_Bound = [Lower_Bound[0] / 4, Lower_Bound[0], Lower_Bound[2]]   
        while scipy.stats.ncf.cdf(F_Statistic, df1, df2, Lower_Bound[2]) > Upper_Limit: Lower_Bound = [Lower_Bound[0], Lower_Bound[2], Lower_Bound[2] + F_Statistic]     
        while Lower_CI_Difference_Value > 0.0000001:
            Lower_Bound = [Lower_Bound[0], (Lower_Bound[0] + Lower_Bound[1]) / 2, Lower_Bound[1]] if scipy.stats.ncf.cdf(F_Statistic, df1, df2, Lower_Bound[1]) < Upper_Limit else [Lower_Bound[1], (Lower_Bound[1] + Lower_Bound[2]) / 2, Lower_Bound[2]]  
            Lower_CI_Difference_Value = abs(scipy.stats.ncf.cdf(F_Statistic, df1, df2, Lower_Bound[1]) - Upper_Limit)        
        return [Lower_Bound[1]]
    
    def Upper_CI(F_Statistic, df1, df2, Lower_Limit, Lower_CI_Difference_Value):
        Upper_Bound = [F_Statistic, 2 * F_Statistic, 3 * F_Statistic]
        while scipy.stats.ncf.cdf(F_Statistic, df1, df2, Upper_Bound[0]) < Lower_Limit:Upper_Bound = [Upper_Bound[0] / 4, Upper_Bound[0], Upper_Bound[2]]
        while scipy.stats.ncf.cdf(F_Statistic, df1, df2, Upper_Bound[2]) > Lower_Limit: Upper_Bound = [Upper_Bound[0], Upper_Bound[2], Upper_Bound[2] + F_Statistic]
        while Lower_CI_Difference_Value > 0.00001: Upper_Bound = [Upper_Bound[0], (Upper_Bound[0] + Upper_Bound[1]) / 2, Upper_Bound[1]] if scipy.stats.ncf.cdf(F_Statistic, df1, df2, Upper_Bound[1]) < Lower_Limit else [Upper_Bound[1], (Upper_Bound[1] + Upper_Bound[2]) / 2, Upper_Bound[2]]; Lower_CI_Difference_Value = abs(scipy.stats.ncf.cdf(F_Statistic, df1, df2, Upper_Bound[1]) - Lower_Limit)
        return [Upper_Bound[1]]
    
    # Calculate lower and upper bounds
    Lower_CI_Final = Lower_CI(F_Statistic, df1, df2, Upper_Limit, Lower_CI_Difference_Value)[0]
    Upper_CI_Final = Upper_CI(F_Statistic, df1, df2, Lower_Limit, Lower_CI_Difference_Value)[0]
    
    return Lower_CI_Final, Upper_CI_Final

# Instalations = this should be installled with the software
#packageNames = ("afex", "emmeans")
#utils = rpackages.importr("utils")
#utils.chooseCRANmirror(ind=1)
#packnames_to_install = [x for x in packageNames if not rpackages.isinstalled(x)]
#utils.install_packages(StrVector(packnames_to_install)) if len(packnames_to_install) > 0 else None

# This Function Will be used later
def Mean_Correlation(data_frame, response_var, independent_var):
    groups = data_frame.groupby(independent_var)[response_var]
    group_combinations = list(combinations(groups, 2))
    correlations = []

    for group1, group2 in group_combinations:
        correlation, _ = pearsonr(group1[1], group2[1])
        correlations.append(correlation)

    return np.mean(correlations)

# Import of R packages
afex = rpackages.importr("afex")
emmeans = rpackages.importr("emmeans")

data1 = {
    'Group': [-1] * 168 + [1] * 174,
    'Target.RT': [697.6445313, 535.0078125, 1042.867188, 508.5755208, 837.9765625, 592.1875, 641.9778646, 800.0507813, 614.40625, 686.3359375, 716.2591146, 715.625, 616.8385417, 773.8671875, 785.4140625, 682.5234375, 594.4088542, 592.6132813, 673.8684896, 724.375, 738.3125, 540.7825521, 583.296875, 660.2877604, 667.3515625, 706.1627604, 508.5143229, 708.6484375, 620.421875, 869.6015625, 729.9375, 742.3229167, 730.546875, 586.5416667, 733.5625, 768.2109375, 788.2929688, 830.4453125, 694.3658854, 653.5625, 606.8111979, 687.3320313, 744.7070313, 734.546875, 645.5234375, 738.4921875, 801.5820313, 654.984375, 819.9088542, 810.1484375, 627.46875, 839.359375, 786.140625, 524.5625, 816.8528646, 663.2890625, 719.9375, 431.34375, 768.8606771, 519.6796875, 738.2226563, 569.296875, 652.125, 762.9765625, 539.1210938, 654.7486979, 698.5, 700.5078125, 653.328125, 808.5572917, 650.1380208, 592.0859375, 610.84375, 591.65625, 637.28125, 635.6992188, 693.5976563, 575.1171875, 556.0664063, 634.515625, 658.859375, 693.375, 504.859375, 562.2578125, 527.859375, 677.6028646, 620.1536458, 717.0859375, 623.1119792, 593.6875, 704.2539063, 562.9348958, 748.4453125, 715.2955729, 632.8984375, 601.796875, 619.578125, 669.65625, 724.4140625, 682.9492188, 560.5755208, 601.7122396, 788.921875, 571.0234375, 840.8203125, 751.2604167, 561.7851563, 724.8815104, 735.5247396, 522.4401042, 812.015625, 589.515625, 712.8828125, 443.984375, 783.0546875, 490.15625, 746.671875, 574.15625, 684.328125, 807.9140625, 560.8125, 666.0859375, 715.828125, 728.1015625, 631.171875, 787.6171875, 684.8125, 609.0390625, 587.203125, 579.15625, 623.34375, 647.46875, 717.890625, 546.328125, 553.75, 655.7890625, 630.3515625, 664.6328125, 484.6015625, 633.0625, 536.921875, 725.9921875, 634.765625, 717.1484375, 638.375, 599.875, 712.328125, 642.203125, 760.6640625, 714.21875, 694.265625, 642.4921875, 595.96875, 687.1953125, 711.7421875, 655.6796875, 611.890625, 650.96875, 770.828125, 583.1875, 808.609375, 762.2890625, 602.125, 718.6484375, 704.2421875, 457.625, 792.8359375, 636.265625, 995.90625, 949.7421875, 1016.257813, 977.5481771, 1203.190104, 973.8359375, 911.5247396, 1083.28776, 927.2890625, 894.7708333, 809.15625, 972.0833333, 908.8671875, 841.6471354, 864.7994792, 1065, 878.9466146, 1017.804688, 1020.960938, 1256.070313, 1128.4375, 1082.765625, 1127.828125, 878.109375, 882.4544271, 1194.78125, 952.1080729, 1268.270833, 886.2591146, 1088.546875, 914.8984375, 991.0338542, 1102.867188, 1151.109375, 789.109375, 1073.1875, 1034.496094, 1075.5, 882.9427083, 1046.65625, 1062.375, 917.75, 1067.63151, 1000.882813, 929.2552083, 717.6575521, 840.4453125, 633.3463542, 927.28125, 935.6875, 1079.289063, 1023.523438, 1151.65625, 1000.109375, 1165.38151, 987.1054688, 956.0664063, 972.3177083, 853.9141743, 810.53125, 898.6888021, 959.046875, 1216.164063, 995.9635417, 894.3828125, 1115.5, 790.8359375, 927.109375, 724.8385417, 988.453125, 905.1640625, 865.9921875, 900.0625, 915.0039063, 855.9765625, 865.8020833, 909.9205729, 1161.001302, 1035.947917, 777.3867188, 956.6888021, 895.7421875, 928.25, 1122.739583, 938.1953125, 946.0104167, 389.34375, 927.7395833, 907.0078125, 985.5703125, 798.0885417, 958.4401042, 782.6067708, 915.9427083, 1023.195313, 917.2356771, 822.1015625, 999.5963542, 1176.9375, 693.3046875, 1064.09375, 1028.695313, 961.1640625, 747.59375, 829.1171875, 664.140625, 781.8046875, 887.453125, 856.2903646, 875.5690104, 874.4023438, 777.8242188, 1077.34375, 996.0234375, 1003.257813, 1047.296875, 873.6046512, 824.796875, 900.984375, 932.6640625, 1231.984375, 1018.203125, 856.5703125, 1163.789063, 823.0234375, 896.640625, 758.4296875, 958.4765625, 831.7421875, 799.5234375, 833.5234375, 962.0859375, 823.2890625, 877.8984375, 905.21875, 1174.859375, 1129.552083, 832.203125, 962.375, 823.453125, 895.46875, 1130.460938, 953.5625, 961.1458333, 382.703125, 1051.03125, 867.59375, 917.4140625, 806.859375, 1010.414063, 799.1328125, 966.765625, 978.09375, 945.0234375, 891.6328125, 1033.578125, 1131.53125, 824.5625, 946.7109375, 998.5625, 894.515625, 708.9609375, 805.5078125, 619.453125, 825.625, 874.5234375, 889.046875, 890.0703125, 953.3984375, 802.7890625, 1184.1875, 973.453125, 933, 1011.804688],
    'Type': [1] * 56 + [2] * 56 + [3] * 56 + [1] * 58 + [2] * 58 + [3] * 58,
    'Subject': [i for i in range(59, 115)] * 3 + [i for i in range(1, 59)] * 3 
}
data1 = pd.DataFrame(data1)

data2 = {
    'Group': [-1] * 12 + [1] * 18,
    'Target.RT': [12, 14, 18, 21, 53, 56 , 52, 14, 13, 16, 5 , 7, 34, 33, 14, 18, 22, 55, 67, 23, 25, 23, 29, 12, 15, 16, 7, 4, 65, 23],
    'Type': [1] * 4 + [2] * 4 + [3] * 4 + [1] * 6 + [2] * 6 + [3] * 6,
    'Subject': [i for i in range(1, 5)] * 3 + [i for i in range(5, 11)] * 3 
}
data2 = pd.DataFrame(data2)

def Two_Way_Mixed(data, dependent_variable, subject_variable, within=None, between=None):
    
    with localconverter(ro.default_converter + pandas2ri.converter): New_Data = ro.conversion.py2rpy(data)   

    # Defining The Model *****DONT FORGET TO CHECK THE CORRECTION OPTION
    Between_Within_Model = afex.aov_ez(id=subject_variable, dv=dependent_variable, between=between, within=within ,data=New_Data)

    # Sum of Squares for the General Model  
    SSBetween = ro.r.summary(Between_Within_Model)[3][1] # type: ignore
    SSWithin = ro.r.summary(Between_Within_Model)[3][2] # type: ignore
    SSInt = ro.r.summary(Between_Within_Model)[3][3] # type: ignore
    SSSubjects = ro.r.summary(Between_Within_Model)[3][8] # type: ignore
    SSInt_Subjects_Within = ro.r.summary(Between_Within_Model)[3][10] # type: ignore
    SSEffect = np.array([SSBetween, SSWithin, SSInt])
    SSError = np.array([SSSubjects, SSInt_Subjects_Within, SSInt_Subjects_Within])

    # General Table and Parameters
    Means_Table = data.groupby(by=[subject_variable] + [f"{var}" for var in between] + within)[dependent_variable].mean().reset_index().rename(columns={dependent_variable: 'Subject_Between_Within_Mean'}) # type: ignore
    Means_Table['Within_Between_Group'] = Means_Table.groupby(by=[f"{var}" for var in between] + within).ngroup() + 1
    Means_Table['Within_Between_Mean'] = Means_Table.groupby('Within_Between_Group')['Subject_Between_Within_Mean'].transform('mean')
    Means_Table["Residuals"] = (Means_Table['Subject_Between_Within_Mean'] - Means_Table['Within_Between_Mean'])**2
    Means_Table['Within_Mean'] = Means_Table.groupby(within)['Subject_Between_Within_Mean'].transform('mean')
    Means_Table['Between_Mean'] = Means_Table.groupby(between)['Subject_Between_Within_Mean'].transform('mean')
    Means_Table['Subject_Mean'] = Means_Table.groupby(subject_variable)['Subject_Between_Within_Mean'].transform('mean')
    Means_Table['Grand_Mean'] = Means_Table['Subject_Between_Within_Mean'].mean()
    Means_Table['SS_Within'] =  (Means_Table['Within_Between_Mean'] - Means_Table['Between_Mean']) **2 #this is for the simple effects
    Means_Table['SS_Int_Within'] = (Means_Table['Subject_Between_Within_Mean'] - Means_Table['Subject_Mean'] - Means_Table['Within_Between_Mean'] + Means_Table['Between_Mean'])**2
    Means_Table['SS_Subjects'] = (Means_Table['Subject_Between_Within_Mean'] - Means_Table['Subject_Mean'])**2 
    Means_Table['SS_Between'] = (Means_Table['Within_Between_Mean'] - Means_Table['Within_Mean'])**2 #this for the simple effects
    Means_Table['SS_Subjects_Within_Between'] = (Means_Table['Subject_Between_Within_Mean'] - Means_Table['Within_Between_Mean'])**2 #This term includes both Error terms of the design (Interaction with subjects + subjects)

    SSTotal = np.sum((Means_Table['Subject_Between_Within_Mean'] - Means_Table['Grand_Mean'])**2)

    # Degrees of Freedom
    Number_Of_Measures = len(Means_Table)
    df_between = ro.r.summary(Between_Within_Model)[3][5] # type: ignore
    df_within = ro.r.summary(Between_Within_Model)[3][6] # type: ignore
    df_int = ro.r.summary(Between_Within_Model)[3][7] # type: ignore
    df_subjects = ro.r.summary(Between_Within_Model)[3][12] # type: ignore
    df_Int_Subjects_Within = ro.r.summary(Between_Within_Model)[3][14] # type: ignore
    df_effect = np.array([df_between, df_within, df_int])
    df_error = np.array([df_subjects, df_Int_Subjects_Within, df_Int_Subjects_Within])
    dftotal = df_between + df_within + df_int + df_subjects + df_Int_Subjects_Within

    # Mean Squares
    MSBetween = SSBetween / df_between
    MSwithin = SSWithin / df_within
    MSint = SSInt / df_int
    MSSubjects = SSSubjects / df_subjects
    MSInt_Subjects_Within = SSInt_Subjects_Within / df_Int_Subjects_Within
    MS_effect = np.array([MSBetween, MSwithin, MSint])
    MS_error = np.array([MSSubjects, MSInt_Subjects_Within, MSInt_Subjects_Within])

    # F values
    F_Statistic_Between_Effect = MSBetween / MSSubjects
    F_Statistic_Within_Effect = MSwithin /  MSInt_Subjects_Within
    F_Statistic_Interaction_Effect = MSint / MSInt_Subjects_Within
    F_values = np.array([F_Statistic_Between_Effect, F_Statistic_Within_Effect, F_Statistic_Interaction_Effect])
    
    # p values
    p_value_Between_Effect = p_value = scipy.stats.f.sf((F_Statistic_Between_Effect), df_between, df_subjects)
    p_value_Within_Effect = scipy.stats.f.sf((F_Statistic_Within_Effect), df_within, df_Int_Subjects_Within)
    p_value_Interaction_Effect = scipy.stats.f.sf((F_Statistic_Interaction_Effect), df_int, df_Int_Subjects_Within)
    p_values = np.array([p_value_Between_Effect, p_value_Within_Effect, p_value_Interaction_Effect])

    # Other Parameters
    Number_Of_Between_Variable_Groups = (df_between+1)
    Number_Of_Within_Variable_Groups = (df_within+1)
    Number_of_Measures = dftotal + 1
    Sample_Size = Number_of_Measures / (df_within + 1)
    Sample_Size_Group = data.groupby(by=[f"{var}" for var in between])[dependent_variable].count().values / Number_Of_Within_Variable_Groups
    Mean_Correlation_Within = Mean_Correlation(Means_Table, 'Subject_Between_Within_Mean', within)
    Grand_mean = Means_Table["Subject_Between_Within_Mean"].mean()
    Means_Between = data.groupby(by=[f"{var}" for var in between])[dependent_variable].mean().reset_index().rename(columns={dependent_variable: 'Mean'}).values
    Means_Within = data.groupby(by=[f"{var}" for var in within])[dependent_variable].mean().reset_index().rename(columns={dependent_variable: 'Mean'}).values
    Means_Between_within = data.groupby(by=[f"{var}" for var in between + within])[dependent_variable].mean().reset_index().rename(columns={dependent_variable: 'Mean'}).values
    SD_Between = data.groupby(by=[f"{var}" for var in between])[dependent_variable].std().values
    SD_Within = data.groupby(by=[f"{var}" for var in within])[dependent_variable].std().values
    SD_Between_Within = data.groupby(by=[f"{var}" for var in between + within])[dependent_variable].std().values
    SE_Between = SD_Between / np.sqrt(data.groupby(by=[f"{var}" for var in between])[dependent_variable].count().values)
    SE_Within = SD_Within / np.sqrt(data.groupby(by=[f"{var}" for var in within])[dependent_variable].count().values)
    SE_Between_Within = SD_Between_Within / np.sqrt(data.groupby(by=[f"{var}" for var in between + within])[dependent_variable].count().values)
    SE_Marginal_Means_Between = np.sqrt((MSSubjects / Number_Of_Within_Variable_Groups) / Sample_Size_Group)
    SE_Marginal_Means_Within =  ((Means_Table.groupby(by = within)['Residuals'].sum().values) / Sample_Size)**0.5 / np.sqrt(Sample_Size - Number_Of_Between_Variable_Groups)
    SE_Marginal_Means_Withih_Between = np.transpose(np.sqrt(np.meshgrid(Sample_Size_Group, Means_Table.groupby(within)["Residuals"].sum().values)/ np.array(df_subjects)/ Sample_Size_Group)[1]).flatten()
    

    # Eta Squares
    Between_Eta_Square = SSBetween / SSTotal
    Within_Eta_Square = SSWithin / SSTotal
    Int_Eta_Square = SSInt / SSTotal
    Eta_Square = np.array([Between_Eta_Square, Within_Eta_Square, Int_Eta_Square])

    # Partial Eta Squares
    Between_Partial_Eta_Square = SSBetween / (SSBetween + SSSubjects)
    Within_Partial_Eta_Square = SSWithin / (SSWithin + SSInt_Subjects_Within)
    Int_Partial_Eta_Square = SSInt / (SSInt + SSInt_Subjects_Within)
    Partial_Eta_Square = np.array([Between_Partial_Eta_Square, Within_Partial_Eta_Square, Int_Partial_Eta_Square])

    # Epsilon Square
    Between_Epsilon_Square = (SSBetween - df_between * MSSubjects) / SSTotal
    Within_Epsilon_Square = (SSWithin - df_within * MSInt_Subjects_Within) / SSTotal
    Int_Epsilon_Square = (SSInt - df_int * MSInt_Subjects_Within) / SSTotal
    Epsilon_Square = np.array([Between_Epsilon_Square, Within_Epsilon_Square, Int_Epsilon_Square])

    # Partial Epsilon Square
    Between_Partial_Epsilon_Square = (SSBetween - df_between * MSSubjects) / (SSBetween + SSSubjects)
    Within_Partial_Epsilon_Square = (SSWithin - df_within * MSInt_Subjects_Within) / (SSWithin + SSInt_Subjects_Within)
    Int_Partial_Epsilon_Square = (SSInt - df_int * MSInt_Subjects_Within) /(SSInt + SSInt_Subjects_Within)
    Partial_Epsilon_Square = np.array([Between_Partial_Epsilon_Square, Within_Partial_Epsilon_Square, Int_Partial_Epsilon_Square])    

    # Omega Square
    Between_Omega_Square = (SSBetween - df_between * MSSubjects) / (SSTotal + MSSubjects)
    Within_Omega_Square = (SSWithin - df_within * MSInt_Subjects_Within) / (SSTotal + MSSubjects)
    Int_Omega_Square = (SSInt - df_int * MSInt_Subjects_Within) / (SSTotal + MSSubjects)
    Omega_Square = np.array([Between_Omega_Square, Within_Omega_Square, Int_Omega_Square])

    # Partial Omega Square (Maxwell and Delaney, 2004)
    Between_Partial_Omega_Square = (SSBetween - df_between * MSSubjects) / (SSBetween + SSSubjects + MSSubjects)
    Within_Partial_Omega_Square = (SSWithin - (df_within * MSInt_Subjects_Within)) / (SSWithin + SSInt_Subjects_Within + SSSubjects + MSSubjects)
    Int_Partial_Omega_Square = (SSInt - (df_int * MSInt_Subjects_Within)) /(SSInt + SSInt_Subjects_Within + SSSubjects + MSSubjects)
    Partial_Omega_Square = np.array([Between_Partial_Omega_Square, Within_Partial_Omega_Square, Int_Partial_Omega_Square])

    # Generalizzed Eta Square
    Between_Generalizzed_Eta_Square = SSBetween / (SSTotal - SSWithin - SSInt)
    Within_Generalizzed_Eta_Square = SSWithin / (SSTotal - SSBetween - SSInt)
    Int_Generalizzed_Eta_Square = SSInt / (SSTotal - SSBetween - SSWithin)
    Generalized_Eta_Square = np.array([Between_Generalizzed_Eta_Square, Within_Generalizzed_Eta_Square, Int_Generalizzed_Eta_Square])

    # Genralized Omega Square Howell
    Between_Generalized_Omega_Square = (SSBetween - df_between * MSInt_Subjects_Within) / ( (SSBetween - df_between*MSSubjects) + (SSInt - df_int * MSInt_Subjects_Within) + Number_Of_Measures * ((SSInt_Subjects_Within+SSSubjects)/ (df_subjects+df_Int_Subjects_Within)))
    Within_Generalized_Omega_Square = (SSWithin - df_within * MSInt_Subjects_Within) / ( (SSWithin - df_within*MSInt_Subjects_Within) + (SSBetween - df_between*MSSubjects) + (SSInt - df_int * MSInt_Subjects_Within) + Number_Of_Measures * ((SSInt_Subjects_Within+SSSubjects)/ (df_subjects+df_Int_Subjects_Within)))
    Int_Generalizzed_Omega_Square = (SSInt - df_int * MSInt_Subjects_Within) / ( (SSBetween - df_between*MSSubjects) + (SSInt - df_int * MSInt_Subjects_Within) + Number_Of_Measures * ((SSInt_Subjects_Within+SSSubjects)/ (df_subjects+df_Int_Subjects_Within)))
    Generalized_Omega_Square = np.array([Between_Generalized_Omega_Square, Within_Generalized_Omega_Square, Int_Generalizzed_Omega_Square])

    # Cohens f^2
    Between_Cohens_f_square = Between_Eta_Square / (1 - Between_Eta_Square)
    Within_Cohens_f_square = Within_Eta_Square / (1 - Within_Eta_Square)
    Int_cohens_f_square = Int_Eta_Square / (1 - Int_Eta_Square)
    Cohens_f_square = np.array([Between_Cohens_f_square, Within_Cohens_f_square, Int_cohens_f_square])

    # Partial Cohens f^2
    Between_Partial_Cohens_f_square = Between_Partial_Eta_Square / (1 - Between_Partial_Eta_Square)
    Within_Partial_Cohens_f_square = Within_Partial_Eta_Square / (1 - Within_Partial_Eta_Square)
    Int_Partial_cohens_f_square = Int_Partial_Eta_Square / (1 - Int_Partial_Eta_Square)
    Partial_Cohens_f_square = np.array([Between_Partial_Cohens_f_square, Within_Partial_Cohens_f_square, Int_Partial_cohens_f_square])
    
    # G-power Effect Sizes
    Between_Gpower_partial_f_square = Between_Partial_Cohens_f_square
    Within_Gpower_Partial_f_square = Within_Partial_Cohens_f_square * ((Sample_Size - 1) / Sample_Size) * ((Number_Of_Within_Variable_Groups - 1) / Number_Of_Within_Variable_Groups) * (1 - Mean_Correlation_Within)
    Int_Gpower_Partial_f_square = Int_Partial_cohens_f_square * ((Sample_Size - Number_Of_Between_Variable_Groups) / Sample_Size) * ((Number_Of_Within_Variable_Groups - 1) / Number_Of_Within_Variable_Groups) * (1 - Mean_Correlation_Within)
    Gpower_Partial_Cohens_f_square = np.array([Between_Gpower_partial_f_square, Within_Gpower_Partial_f_square, Int_Gpower_Partial_f_square])

    # R sqaure and Adjustd R square
    R_Square_Model = 1 - (dftotal / (df_subjects + df_Int_Subjects_Within)) * (SSInt_Subjects_Within / SSTotal)
    R_Square_Adjusted_Model = 1 - (dftotal / (df_Int_Subjects_Within)) * (SSInt_Subjects_Within / SSTotal)

    # Confidence Intervals for the Effect Size
    Between_Effect_Lower_NCP_CI, Between_Effect_Upper_NCP_CI = Non_Central_CI_F(F_Statistic_Between_Effect, df_between, df_subjects, confidence_level)
    Within_Effect_Lower_NCP_CI, Within_Effect_Upper_NCP_CI = Non_Central_CI_F(F_Statistic_Within_Effect, df_within, df_Int_Subjects_Within, confidence_level)
    Interaction_Effect_Lower_NCP_CI, Interaction_Upper_NCP_CI = Non_Central_CI_F(F_Statistic_Interaction_Effect, df_int, df_Int_Subjects_Within, confidence_level)

    # Confidence Intervals for Partial Eta Square
    Partial_Eta_CI_Between_Effect_Lower = Between_Effect_Lower_NCP_CI / (Between_Effect_Lower_NCP_CI + df_subjects + df_between + 1) 
    Partial_Eta_CI_Between_Effect_Upper = Between_Effect_Upper_NCP_CI / (Between_Effect_Upper_NCP_CI + df_subjects + df_between + 1) 
    Partial_Eta_CI_Within_Effect_Lower = Within_Effect_Lower_NCP_CI / (Within_Effect_Lower_NCP_CI + df_Int_Subjects_Within + df_within + 1) 
    Partial_Eta_CI_Within_Effect_Upper = Within_Effect_Upper_NCP_CI / (Within_Effect_Upper_NCP_CI + df_Int_Subjects_Within + df_within + 1) 
    Partial_Eta_CI_Interaction_Effect_Lower = Interaction_Effect_Lower_NCP_CI / (Interaction_Effect_Lower_NCP_CI + df_Int_Subjects_Within + df_int + 1) 
    Partial_Eta_CI_Interaction_Effect_Upper = Interaction_Upper_NCP_CI / (Interaction_Upper_NCP_CI + df_Int_Subjects_Within + df_int + 1) 

    # Confidence Intervals for Eta Square
    F_from_Effect_Size_Between = ((Between_Eta_Square / df_between) / ((1 - Between_Eta_Square) / df_subjects))
    F_from_Effect_Size_Within = ((Within_Eta_Square / df_within) / ((1 - Within_Eta_Square) / df_Int_Subjects_Within))
    F_from_Effect_Size_Interaction = ((Int_Eta_Square / df_int) / ((1 - Int_Eta_Square) / df_Int_Subjects_Within))

    Between_Effect_Lower_CI_Eta, Between_Effect_Upper_CI_Eta = Non_Central_CI_F(F_from_Effect_Size_Between, df_between, df_subjects, confidence_level)
    Within_Effect_Lower_CI_Eta, Within_Effect_Upper_CI_Eta = Non_Central_CI_F(F_from_Effect_Size_Within, df_within, df_Int_Subjects_Within, confidence_level)
    Interaction_Effect_Lower_CI_Eta, Interaction_Upper_CI_Eta = Non_Central_CI_F(F_from_Effect_Size_Interaction, df_int, df_Int_Subjects_Within, confidence_level)

    Eta_CI_Between_Effect_Lower = Between_Effect_Lower_CI_Eta / (Between_Effect_Lower_CI_Eta + df_subjects) 
    Eta_CI_Between_Effect_Upper = Between_Effect_Upper_CI_Eta / (Between_Effect_Upper_CI_Eta + df_subjects) 
    Eta_CI_Within_Effect_Lower = Within_Effect_Lower_CI_Eta / (Within_Effect_Lower_CI_Eta + df_Int_Subjects_Within) 
    Eta_CI_Within_Effect_Upper = Within_Effect_Upper_CI_Eta / (Within_Effect_Upper_CI_Eta + df_Int_Subjects_Within) 
    Eta_CI_Interaction_Effect_Lower = Interaction_Effect_Lower_CI_Eta / (Interaction_Effect_Lower_CI_Eta + df_Int_Subjects_Within) 
    Eta_CI_Interaction_Effect_Upper = Interaction_Upper_CI_Eta / (Interaction_Upper_CI_Eta + df_Int_Subjects_Within) 

    # Confidence Intervals for Generalized Eta Square
    F_from_Generalized_Eta_Between = ((Between_Generalizzed_Eta_Square / df_between) / ((1 - Between_Generalizzed_Eta_Square) / df_subjects))
    F_from_Generalized_Eta_Within = ((Within_Generalizzed_Eta_Square / df_within) / ((1 - Within_Generalizzed_Eta_Square) / df_Int_Subjects_Within))
    F_from_Generalized_Eta_Interaction = ((Int_Generalizzed_Eta_Square / df_int) / ((1 - Int_Generalizzed_Eta_Square) / df_Int_Subjects_Within))

    Between_Effect_Lower_NCP_CI_Eta_Generalized, Between_Effect_Upper_CI_Eta_Generalized = Non_Central_CI_F(F_from_Generalized_Eta_Between, df_between, df_subjects, confidence_level)
    Within_Effect_Lower_CI_Eta_Generalized, Within_Effect_Upper_CI_Eta_Generalized = Non_Central_CI_F(F_from_Generalized_Eta_Within, df_within, df_Int_Subjects_Within, confidence_level)
    Interaction_Effect_Lower_CI_Eta_Generalized, Interaction_Upper_CI_Eta_Generalized = Non_Central_CI_F(F_from_Generalized_Eta_Interaction, df_int, df_Int_Subjects_Within, confidence_level)

    Generalized_Eta_CI_Between_Effect_Lower = Between_Effect_Lower_NCP_CI_Eta_Generalized / (Between_Effect_Lower_NCP_CI_Eta_Generalized + df_subjects) 
    Generalized_Eta_CI_Between_Effect_Upper = Between_Effect_Upper_CI_Eta_Generalized / (Between_Effect_Upper_CI_Eta_Generalized + df_subjects) 
    Generalized_Eta_CI_Within_Effect_Lower = Within_Effect_Lower_CI_Eta_Generalized / (Within_Effect_Lower_CI_Eta_Generalized + df_Int_Subjects_Within) 
    Generalized_Eta_CI_Within_Effect_Upper = Within_Effect_Upper_CI_Eta_Generalized / (Within_Effect_Upper_CI_Eta_Generalized + df_Int_Subjects_Within) 
    Generalized_Eta_CI_Interaction_Effect_Lower = Interaction_Effect_Lower_CI_Eta_Generalized / (Interaction_Effect_Lower_CI_Eta_Generalized + df_Int_Subjects_Within) 
    Generalized_Eta_CI_Interaction_Effect_Upper = Interaction_Upper_CI_Eta_Generalized / (Interaction_Upper_CI_Eta_Generalized + df_Int_Subjects_Within) 

    # Confidence Intervals for Epsilon
    F_from_Epsilon_Between = ((Between_Epsilon_Square / df_between) / ((1 - Between_Epsilon_Square) / df_subjects))
    F_from_Epsilon_Within = ((Within_Epsilon_Square / df_within) / ((1 - Within_Epsilon_Square) / df_Int_Subjects_Within))
    F_from_Epsilon_Interaction = ((Int_Epsilon_Square / df_int) / ((1 - Int_Epsilon_Square) / df_Int_Subjects_Within))

    Between_Effect_Lower_NCP_CI_Epsilon, Between_Effect_Upper_Epsilon = Non_Central_CI_F(F_from_Epsilon_Between, df_between, df_subjects, confidence_level)
    Within_Effect_Lower_CI_Epsilon, Within_Effect_Upper_CI_Epsilon = Non_Central_CI_F(F_from_Epsilon_Within, df_within, df_Int_Subjects_Within, confidence_level)
    Interaction_Effect_Lower_CI_Epsilon, Interaction_Upper_CI_Epsilon = Non_Central_CI_F(F_from_Epsilon_Interaction, df_int, df_Int_Subjects_Within, confidence_level)

    Epsilon_CI_Between_Effect_Lower = Between_Effect_Lower_NCP_CI_Epsilon / (Between_Effect_Lower_NCP_CI_Epsilon + df_subjects) 
    Epsilon_CI_Between_Effect_Upper = Between_Effect_Upper_Epsilon / (Between_Effect_Upper_Epsilon + df_subjects) 
    Epsilon_CI_Within_Effect_Lower = Within_Effect_Lower_CI_Epsilon / (Within_Effect_Lower_CI_Epsilon + df_Int_Subjects_Within) 
    Epsilon_CI_Within_Effect_Upper = Within_Effect_Upper_CI_Epsilon / (Within_Effect_Upper_CI_Epsilon + df_Int_Subjects_Within) 
    Epsilon_CI_Interaction_Effect_Lower = Interaction_Effect_Lower_CI_Epsilon / (Interaction_Effect_Lower_CI_Epsilon + df_Int_Subjects_Within) 
    Epsilon_CI_Interaction_Effect_Upper = Interaction_Upper_CI_Epsilon / (Interaction_Upper_CI_Epsilon + df_Int_Subjects_Within) 

    # Confidence Intervals for Partial Epsilon Square
    F_from_Partial_Epsilon_Between = ((Between_Partial_Epsilon_Square / df_between) / ((1 - Between_Partial_Epsilon_Square) / df_subjects))
    F_from_Partial_Epsilon_Within = ((Within_Partial_Epsilon_Square / df_within) / ((1 - Within_Partial_Epsilon_Square) / df_Int_Subjects_Within))
    F_from_Partial_Epsilon_Interaction = ((Int_Partial_Epsilon_Square / df_int) / ((1 - Int_Partial_Epsilon_Square) / df_Int_Subjects_Within))

    Between_Effect_Lower_NCP_CI_Partial_Epsilon, Between_Effect_Upper_Partial_Epsilon = Non_Central_CI_F(F_from_Partial_Epsilon_Between, df_between, df_subjects, confidence_level)
    Within_Effect_Lower_CI_Partial_Epsilon, Within_Effect_Upper_CI_Partial_Epsilon = Non_Central_CI_F(F_from_Partial_Epsilon_Within, df_within, df_Int_Subjects_Within, confidence_level)
    Interaction_Effect_Lower_CI_Partial_Epsilon, Interaction_Upper_CI_Partial_Epsilon = Non_Central_CI_F(F_from_Partial_Epsilon_Interaction, df_int, df_Int_Subjects_Within, confidence_level)

    Partial_Epsilon_CI_Between_Effect_Lower = Between_Effect_Lower_NCP_CI_Partial_Epsilon / (Between_Effect_Lower_NCP_CI_Partial_Epsilon + df_subjects) 
    Partial_Epsilon_CI_Between_Effect_Upper = Between_Effect_Upper_Partial_Epsilon / (Between_Effect_Upper_Partial_Epsilon + df_subjects) 
    Partial_Epsilon_CI_Within_Effect_Lower = Within_Effect_Lower_CI_Partial_Epsilon / (Within_Effect_Lower_CI_Partial_Epsilon + df_Int_Subjects_Within) 
    Partial_Epsilon_CI_Within_Effect_Upper = Within_Effect_Upper_CI_Partial_Epsilon / (Within_Effect_Upper_CI_Partial_Epsilon + df_Int_Subjects_Within) 
    Partial_Epsilon_CI_Interaction_Effect_Lower = Interaction_Effect_Lower_CI_Partial_Epsilon / (Interaction_Effect_Lower_CI_Partial_Epsilon + df_Int_Subjects_Within) 
    Partial_Epsilon_CI_Interaction_Effect_Upper = Interaction_Upper_CI_Partial_Epsilon / (Interaction_Upper_CI_Partial_Epsilon + df_Int_Subjects_Within) 

    # Confidence Intervals for Patial Omega Square
    F_from_Partial_Omega_Between = ((Between_Partial_Omega_Square / df_between) / ((1 - Between_Partial_Omega_Square) / df_subjects))
    F_from_Partial_Omega_Within = ((Within_Partial_Omega_Square / df_within) / ((1 - Within_Partial_Omega_Square) / df_Int_Subjects_Within))
    F_from_Partial_Omega_Interaction = ((Int_Partial_Omega_Square / df_int) / ((1 - Int_Partial_Omega_Square) / df_Int_Subjects_Within))

    Between_Effect_Lower_NCP_CI_Partial_Omega, Between_Effect_Upper_Partial_Omega = Non_Central_CI_F(F_from_Partial_Omega_Between, df_between, df_subjects, confidence_level)
    Within_Effect_Lower_CI_Partial_Omega, Within_Effect_Upper_CI_Partial_Omega = Non_Central_CI_F(F_from_Partial_Omega_Within, df_within, df_Int_Subjects_Within, confidence_level)
    Interaction_Lower_CI_Partial_Omega, Interaction_Upper_CI_Partial_Omega = Non_Central_CI_F(F_from_Partial_Omega_Interaction, df_int, df_Int_Subjects_Within, confidence_level)

    Partial_Omega_CI_Between_Effect_Lower = Between_Effect_Lower_NCP_CI_Partial_Omega / (Between_Effect_Lower_NCP_CI_Partial_Omega + df_subjects) 
    Partial_Omega_CI_Between_Effect_Upper = Between_Effect_Upper_Partial_Omega / (Between_Effect_Upper_Partial_Omega + df_subjects) 
    Partial_Omega_CI_Within_Effect_Lower = Within_Effect_Lower_CI_Partial_Omega / (Within_Effect_Lower_CI_Partial_Omega + df_Int_Subjects_Within) 
    Partial_Omega_CI_Within_Effect_Upper = Within_Effect_Upper_CI_Partial_Omega / (Within_Effect_Upper_CI_Partial_Omega + df_Int_Subjects_Within) 
    Partial_Omega_CI_Interaction_Effect_Lower = Interaction_Lower_CI_Partial_Omega / (Interaction_Lower_CI_Partial_Omega + df_Int_Subjects_Within) 
    Partial_Omega_CI_Interaction_Effect_Upper = Interaction_Upper_CI_Partial_Omega / (Interaction_Upper_CI_Partial_Omega + df_Int_Subjects_Within)

    # Confidence Intervals for Omega Square
    F_from_Omega_Between = ((Between_Omega_Square / df_between) / ((1 - Between_Omega_Square) / df_subjects))
    F_from_Omega_Within = ((Within_Omega_Square / df_within) / ((1 - Within_Omega_Square) / df_Int_Subjects_Within))
    F_from_Omega_Interaction = ((Int_Omega_Square / df_int) / ((1 - Int_Omega_Square) / df_Int_Subjects_Within))

    Between_Effect_Lower_NCP_CI_Omega, Between_Effect_Upper_Omega = Non_Central_CI_F(F_from_Omega_Between, df_between, df_subjects, confidence_level)
    Within_Effect_Lower_CI_Omega, Within_Effect_Upper_CI_Omega = Non_Central_CI_F(F_from_Omega_Within, df_within, df_Int_Subjects_Within, confidence_level)
    Interaction_Lower_CI_Omega, Interaction_Upper_CI_Omega = Non_Central_CI_F(F_from_Omega_Interaction, df_int, df_Int_Subjects_Within, confidence_level)

    Omega_CI_Between_Effect_Lower = Between_Effect_Lower_NCP_CI_Omega / (Between_Effect_Lower_NCP_CI_Omega + df_subjects) 
    Omega_CI_Between_Effect_Upper = Between_Effect_Upper_Omega / (Between_Effect_Upper_Omega + df_subjects) 
    Omega_CI_Within_Effect_Lower = Within_Effect_Lower_CI_Omega / (Within_Effect_Lower_CI_Omega + df_Int_Subjects_Within) 
    Omega_CI_Within_Effect_Upper = Within_Effect_Upper_CI_Omega / (Within_Effect_Upper_CI_Omega + df_Int_Subjects_Within) 
    Omega_CI_Interaction_Effect_Lower = Interaction_Lower_CI_Omega / (Interaction_Lower_CI_Omega + df_Int_Subjects_Within) 
    Omega_CI_Interaction_Effect_Upper = Interaction_Upper_CI_Omega / (Interaction_Upper_CI_Omega + df_Int_Subjects_Within)

    # Confidence Intervals for Generalized Omega Square
    F_from_Generalized_Omega_Between = ((Between_Generalized_Omega_Square / df_between) / ((1 - Between_Generalized_Omega_Square) / df_subjects))
    F_from_Generalized_Omega_Within = ((Within_Generalized_Omega_Square / df_within) / ((1 - Within_Generalized_Omega_Square) / df_Int_Subjects_Within))
    F_from_Generalized_Omega_Interaction = ((Int_Generalizzed_Omega_Square / df_int) / ((1 - Int_Generalizzed_Omega_Square) / df_Int_Subjects_Within))

    Between_Effect_Lower_NCP_CI_Generalized_Omega, Between_Effect_Upper_Generalized_Omega = Non_Central_CI_F(F_from_Generalized_Omega_Between, df_between, df_subjects, confidence_level)
    Within_Effect_Lower_CI_Generalized_Omega, Within_Effect_Upper_CI_Generalized_Omega = Non_Central_CI_F(F_from_Generalized_Omega_Within, df_within, df_Int_Subjects_Within, confidence_level)
    Interaction_Lower_CI_Generalized_Omega, Interaction_Upper_CI_Generalized_Omega = Non_Central_CI_F(F_from_Generalized_Omega_Interaction, df_int, df_Int_Subjects_Within, confidence_level)

    Generalized_Omega_CI_Between_Effect_Lower = Between_Effect_Lower_NCP_CI_Omega / (Between_Effect_Lower_NCP_CI_Generalized_Omega + df_subjects) 
    Generalized_Omega_CI_Between_Effect_Upper = Between_Effect_Upper_Generalized_Omega / (Between_Effect_Upper_Generalized_Omega + df_subjects) 
    Generalized_Omega_CI_Within_Effect_Lower = Within_Effect_Lower_CI_Generalized_Omega / (Within_Effect_Lower_CI_Generalized_Omega + df_Int_Subjects_Within) 
    Generalized_Omega_CI_Within_Effect_Upper = Within_Effect_Upper_CI_Generalized_Omega / (Within_Effect_Upper_CI_Generalized_Omega + df_Int_Subjects_Within) 
    Generalized_Omega_CI_Interaction_Effect_Lower = Interaction_Lower_CI_Generalized_Omega / (Interaction_Lower_CI_Generalized_Omega + df_Int_Subjects_Within) 
    Generalized_Omega_CI_Interaction_Effect_Upper = Interaction_Upper_CI_Generalized_Omega / (Interaction_Upper_CI_Generalized_Omega + df_Int_Subjects_Within)

    
    # Tets - SJstats
    A = (SSBetween - df_between * MSSubjects) /  (SSTotal + MSSubjects)
    Cohens_f = np.sqrt( (df_between / Number_Of_Measures) * (F_Statistic_Between_Effect - 1))
    Cohens_f2 = np.sqrt(Between_Eta_Square / (1-Between_Eta_Square))
    Cohens_f3 = np.sqrt(Between_Partial_Eta_Square / (1-Between_Partial_Eta_Square))
    A = np.array([Cohens_f, Cohens_f2, Cohens_f3])

    # Contrats of the Withihn Subject Variable
    Within_Contrasts = (ro.r.summary(emmeans.emmeans(Between_Within_Model, specs = within, contr = "pairwise"))) # type: ignore
    Within_Contrasts_Comparison = Within_Contrasts[1][0]
    Within_Mean_Difference = Within_Contrasts[1][1]
    Within_Contrasts_Standard_Error = Within_Contrasts[1][2]
    Within_Contrasts_Degrees_of_freedom_Denominator = Within_Contrasts[1][3]
    Within_Contrasts_Degrees_of_freedom_Numerator = np.repeat(1, len(Within_Contrasts_Comparison))
    Within_Contrasts_t_values = Within_Contrasts[1][4]
    Within_Contrasts_F_values = np.square(Within_Contrasts_t_values)
    Within_Contrast_MSerror = (np.square(Within_Contrasts_Standard_Error) * 2 * np.prod(Sample_Size_Group)) / np.sum(Sample_Size)
    Within_Contrasts_SSerror = Within_Contrast_MSerror * Within_Contrasts_Degrees_of_freedom_Denominator
    Within_Contrasts_Ms_Contrast = Within_Contrasts_F_values * Within_Contrast_MSerror
    Within_Contrasts_SS_Contrast = Within_Contrasts_Ms_Contrast

    # Contrasts of the Between Subject Variable
    Between_Contrasts = (ro.r.summary(emmeans.emmeans(Between_Within_Model, specs = between, contr = "pairwise"))) # type: ignore
    Between_Contrasts_Comparison = Between_Contrasts[1][0]
    Between_Mean_Difference = Between_Contrasts[1][1]
    Between_Contrasts_Standard_Error = Between_Contrasts[1][2]
    Between_Contrasts_Degrees_of_freedom_Denominator = Between_Contrasts[1][3]
    Between_Contrasts_Degrees_of_freedom_Numerator = np.repeat(1, len(Between_Contrasts_Comparison))
    Between_Contrasts_t_values = Between_Contrasts[1][4]
    Between_Contrasts_F_values = np.square(Between_Contrasts_t_values)
    Between_Contrast_MSerror = np.repeat(MSSubjects, len(Between_Contrasts_Comparison))
    Between_Contrasts_SSerror = Between_Contrast_MSerror * Between_Contrasts_Degrees_of_freedom_Denominator
    Between_Contrasts_Ms_Contrast = Between_Contrasts_F_values * Between_Contrast_MSerror
    Between_Contrasts_SS_Contrast = Between_Contrasts_Ms_Contrast
    Between_Contrasts_sample_sizes = list(combinations(Sample_Size_Group, 2))


    # Within in Between Simple Main Effect
    degrees_of_freedom_simple_effect = df_within 
    degrees_of_freedom_simple_error1 = Sample_Size - df_within - 1
    degrees_of_freedom_simple_error2 = Sample_Size - df_within # as in Joint_Test

    # Method 1 - Multivariate Method
    Within_in_Between_Simple_Effect = ((emmeans.joint_tests(Between_Within_Model, by = between[0]))) # type: ignore
    F_Within_In_Between_multivarite_2 = np.array(Within_in_Between_Simple_Effect[4])
    Within_in_Between_Simple_Effect_Wilks_1 = 1 / ((F_Within_In_Between_multivarite_2) / (degrees_of_freedom_simple_error2/degrees_of_freedom_simple_effect) + 1)
    Within_in_Between_Simple_Effect_Wilks_2 =  1 / ((F_Within_In_Between_multivarite_2) / (degrees_of_freedom_simple_error1/degrees_of_freedom_simple_effect) + 1)
    F_Within_In_Between_multivarite_1 = ((1 - Within_in_Between_Simple_Effect_Wilks_1) / Within_in_Between_Simple_Effect_Wilks_1) * (degrees_of_freedom_simple_error1 / degrees_of_freedom_simple_effect)

    Partial_Eta_Square_within_by_between_Multivariate1 = (F_Within_In_Between_multivarite_1 * df_within) / (F_Within_In_Between_multivarite_1 * degrees_of_freedom_simple_effect + degrees_of_freedom_simple_error1)
    Partial_Omega_Sqaure_within_by_between_Multivariate1 = ((F_Within_In_Between_multivarite_1-1) * df_within) / (F_Within_In_Between_multivarite_1 * degrees_of_freedom_simple_effect + degrees_of_freedom_simple_error1 + 1)
    Partial_Epsilon_Square_within_by_between_Multivariate1 = ((F_Within_In_Between_multivarite_1-1) * df_within) / (F_Within_In_Between_multivarite_1 * degrees_of_freedom_simple_effect + degrees_of_freedom_simple_error1)

    Partial_Eta_Square_within_by_between_Multivariate2 = (F_Within_In_Between_multivarite_2 * df_within) / (F_Within_In_Between_multivarite_2 * degrees_of_freedom_simple_effect + degrees_of_freedom_simple_error1)
    Partial_Omega_Sqaure_within_by_between_Multivariate2 = ((F_Within_In_Between_multivarite_2-1) * df_within) / (F_Within_In_Between_multivarite_2 * degrees_of_freedom_simple_effect + degrees_of_freedom_simple_error1 + 1)
    Partial_Epsilon_Square_within_by_between_Multivariate2 = ((F_Within_In_Between_multivarite_2-1) * df_within) / (F_Within_In_Between_multivarite_2 * degrees_of_freedom_simple_effect + degrees_of_freedom_simple_error1)

    Partial_Cohens_f_Multivariate1 = Partial_Eta_Square_within_by_between_Multivariate1 / (1 - Partial_Eta_Square_within_by_between_Multivariate1)
    Partial_Cohens_f_Multivariate2 = Partial_Eta_Square_within_by_between_Multivariate2 / (1 - Partial_Eta_Square_within_by_between_Multivariate2)

    # Method 2 - Univariate Method
    SS_Effect_Within_By_Between = Means_Table.groupby(between)['SS_Within'].sum()
    SS_Error_Within_By_Between = Means_Table.groupby(between)['SS_Int_Within'].sum()
    SS_Subjects_Simple_Effect = Means_Table.groupby(between)['SS_Subjects'].sum()

    Ms_Effect_Within_By_Between = (SS_Effect_Within_By_Between / df_within)
    Ms_Effect_Within_By_Between_Univariate = (SS_Effect_Within_By_Between / df_within)
    Ms_Error_Within_By_Between_Univariate = MSInt_Subjects_Within
    F_Within_in_Between_Univariate =  Ms_Effect_Within_By_Between_Univariate / Ms_Error_Within_By_Between_Univariate

    Partial_Eta_Square_within_by_between_Univariate = SS_Effect_Within_By_Between / (SS_Effect_Within_By_Between + SSInt_Subjects_Within)
    Partial_Omega_Sqaure_within_by_between_Univariate = df_within * (Ms_Effect_Within_By_Between - Ms_Error_Within_By_Between_Univariate) / (SS_Effect_Within_By_Between + SS_Error_Within_By_Between + SS_Subjects_Simple_Effect + MSSubjects )
    Partial_Epsilon_Square_within_by_between_Univariate = df_within * (Ms_Effect_Within_By_Between - Ms_Error_Within_By_Between_Univariate) / (SS_Effect_Within_By_Between + SSInt_Subjects_Within)
    Partial_Cohens_f2_within_by_between_Univariate  = Partial_Eta_Square_within_by_between_Univariate / (1 - Partial_Eta_Square_within_by_between_Univariate)
    
    Eta_Square_within_by_between_Univariate = SS_Effect_Within_By_Between / (SSTotal)
    Omega_Square_within_by_between_Univariate  = df_within * (Ms_Effect_Within_By_Between - Ms_Error_Within_By_Between_Univariate) / (SSTotal + Ms_Error_Within_By_Between_Univariate)  
    Epsilon_Square_within_by_between_Univariate = (df_within * (Ms_Effect_Within_By_Between - Ms_Error_Within_By_Between_Univariate)) / (SSTotal)     
    Cohens_f_Square_within_by_between_Univariate = Eta_Square_within_by_between_Univariate / (1 - Eta_Square_within_by_between_Univariate)
    
    # Method 3 - Use Seperate Anova's
    SS_Subjects_Within_By_Between = Means_Table.groupby(between)['SS_Int_Within'].sum()
    SS_total_Within_By_Between = SS_Effect_Within_By_Between + SS_Error_Within_By_Between + SS_Subjects_Within_By_Between
    Degrees_Of_Freedom_Error_Simple_Seperate = (Sample_Size_Group - 1) * df_within
    Ms_Error_Within_By_Between_Seperate = (SS_Error_Within_By_Between / Degrees_Of_Freedom_Error_Simple_Seperate)
    MS_Subjects_By_Between = SS_Subjects_Within_By_Between / (Degrees_Of_Freedom_Error_Simple_Seperate)
    F_Within_in_Between_Seperate =  Ms_Effect_Within_By_Between / Ms_Error_Within_By_Between_Seperate

    # Effect Sizes
    Eta_Square_within_by_between_seperate = SS_Effect_Within_By_Between / (SSTotal)
    Omega_Square_within_by_between_seperate = df_within * (Ms_Effect_Within_By_Between - Ms_Error_Within_By_Between_Seperate) / (SS_total_Within_By_Between + MS_Subjects_By_Between)  
    Epsilon_Square_within_by_between_seperate = df_within * (Ms_Effect_Within_By_Between - Ms_Error_Within_By_Between_Seperate) / (SS_total_Within_By_Between) 
    Cohens_f_Square_within_by_between_seperate = Eta_Square_within_by_between_seperate / (1 - Eta_Square_within_by_between_seperate)
    
    Partial_Eta_Square_within_by_between_seperate = SS_Effect_Within_By_Between / (SS_Effect_Within_By_Between + SS_Error_Within_By_Between)
    Partial_Omega_Sqaure_within_by_between_seperate = df_within * (Ms_Effect_Within_By_Between - Ms_Error_Within_By_Between_Seperate) / (df_within * Ms_Effect_Within_By_Between + (Sample_Size_Group - Number_Of_Within_Variable_Groups + 1) * Ms_Error_Within_By_Between_Seperate)
    Partial_Epsilon_Square_within_by_between_seperate = df_within * (Ms_Effect_Within_By_Between - Ms_Error_Within_By_Between_Seperate) / (SS_Effect_Within_By_Between + SS_Error_Within_By_Between)
    Partial_Cohens_f2_within_by_between_seperate = Partial_Eta_Square_within_by_between_seperate / (1 - Partial_Eta_Square_within_by_between_seperate)

    # Between in Within Simple Main Effect
    Between_Simple_Effect = ((emmeans.joint_tests(Between_Within_Model, by = within[0]))) # type: ignore
    SS_Effect_Between_in_Within = Means_Table.groupby(within)['SS_Between'].sum()
    MS_Effect_Between_in_Within = SS_Effect_Between_in_Within / df_between
    SS_Error_Between_in_Within = Means_Table.groupby(within)['SS_Subjects_Within_Between'].sum()
    MS_Error_Between_in_Within = SS_Error_Between_in_Within / df_subjects
    F_values_Between_in_Within = MS_Effect_Between_in_Within/MS_Error_Between_in_Within
    p_values_Between_in_Within = Between_Simple_Effect[5]

    Eta_Square_Between_in_Within = SS_Effect_Between_in_Within / (SSTotal) 
    Omega_Square_Between_in_Within = (SS_Effect_Between_in_Within - df_between * MS_Error_Between_in_Within) / (MS_Error_Between_in_Within + SSTotal)
    Epsilon_Square_Between_in_Within = (SS_Effect_Between_in_Within - df_between * MS_Error_Between_in_Within) / SSTotal
    Cohens_f_Between_in_within = Eta_Square_Between_in_Within / (1- Eta_Square_Between_in_Within)
    
    Partial_Eta_Square_Between_in_Within = SS_Effect_Between_in_Within / (SS_Effect_Between_in_Within + SS_Error_Between_in_Within) 
    Partial_Omega_Square_Between_in_Within = (SS_Effect_Between_in_Within - df_between * MS_Error_Between_in_Within)/ ((df_between * MS_Effect_Between_in_Within) + (Sample_Size - df_between) * MS_Error_Between_in_Within)
    Partial_Epsilon_Square_Between_in_Within = (SS_Effect_Between_in_Within - df_between * MS_Error_Between_in_Within) / (SS_Effect_Between_in_Within + SS_Error_Between_in_Within)
    Partial_Cohens_f = SS_Effect_Between_in_Within / (SS_Effect_Between_in_Within + SS_Error_Between_in_Within) 

    

    results = {}

    results["Effect Sum of Squares Deviations"] = SSEffect
    results["Error Sum of Squares Deviations"] = SSError
    results["Effect Degrees of Freedom"] = df_effect
    results["Error Degrees of Freedom"] = df_error
    results["Effect Mean Squares Deviations"] = MS_effect
    results["Error Mean Squares Deviations"] = MS_error
    results["SStotal"] = SSTotal
    results["F-Value"] = F_values
    results["p-values"] = p_values

    results["Eta Square                  "] = Eta_Square
    results["Epsilon Square              "] = Epsilon_Square
    results["Omega Square                "] = Omega_Square
    results["Generalized Eta Square      "] = Generalized_Eta_Square
    results["Generalized Omega Square    "] = Generalized_Omega_Square

    results["Partial Eta Square          "] = Partial_Eta_Square
    results["Partial Epsilon Square      "] = Partial_Epsilon_Square
    results["Partial Omega Square        "] = Partial_Omega_Square
    results["Mean Correaltion"] = Mean_Correlation_Within
    results["Grand Mean"] = Grand_mean
    results["Cohens F Square                "] = Cohens_f_square
    results["Partial Cohens F Square        "] = Partial_Cohens_f_square
    results["G*power Partial Cohens F Square"] = Gpower_Partial_Cohens_f_square
    results["Cohens F                 "] = np.sqrt(Cohens_f_square)
    results["Partial Cohens F         "] = np.sqrt(Partial_Cohens_f_square)
    results["G*power Partial Cohens F (Gpower Dfault) "] = np.sqrt(Gpower_Partial_Cohens_f_square)
    results["Means Between "] = Means_Between
    results["Means Within "] = Means_Within
    results["Means Between_Within "] = Means_Between_within
    results["Standard Deviation Between "] = SD_Between
    results["Standard Deviation Within "] = SD_Within
    results["Standard Deviation Between Within "] = SD_Between_Within
    results["Standard Error Between "] = SE_Between
    results["Standard Error Within "] = SE_Within
    results["Standard Error Between_Within Means "] = SE_Between_Within
    results["Groups Sample Sizes "] = Sample_Size_Group
    results["Within Variable Contrasts table "] = Within_Contrasts
    results["Standard Errors Between Residuals "]  = SE_Marginal_Means_Between
    results["Standard Errors Within Residuals "]  = SE_Marginal_Means_Within
    results["Standard Errors Beatween_Within Means Residuals "]  = SE_Marginal_Means_Withih_Between
    
    # Confidence Intervals
    results["CI Partial Eta Square Between Effect"]  =  np.array([Partial_Eta_CI_Between_Effect_Lower, Partial_Eta_CI_Between_Effect_Upper])
    results["CI Partial Eta Square Within Effect"]  =  np.array([Partial_Eta_CI_Within_Effect_Lower, Partial_Eta_CI_Within_Effect_Upper])
    results["CI Partial Eta Square Interaction Effect"]  =  np.array([Partial_Eta_CI_Interaction_Effect_Lower, Partial_Eta_CI_Interaction_Effect_Upper])
    results["CI Eta Square Between Effect"]  =  np.array([Eta_CI_Between_Effect_Lower, Eta_CI_Between_Effect_Upper])
    results["CI Eta Square Within Effect"]  =  np.array([Eta_CI_Within_Effect_Lower, Eta_CI_Within_Effect_Upper])
    results["CI Eta Square Interaction Effect"]  =  np.array([Eta_CI_Interaction_Effect_Lower, Eta_CI_Interaction_Effect_Upper])
    results["CI Generalized Eta Square Between Effect"]  =  np.array([Generalized_Eta_CI_Between_Effect_Lower, Generalized_Eta_CI_Between_Effect_Upper])
    results["CI Generalized Eta Square Within Effect"]  =  np.array([Generalized_Eta_CI_Within_Effect_Lower, Generalized_Eta_CI_Within_Effect_Upper])
    results["CI Generalized Eta Square Interaction Effect"]  =  np.array([Generalized_Eta_CI_Interaction_Effect_Lower, Generalized_Eta_CI_Interaction_Effect_Upper])
    results["CI Epsilon Square Between Effect"]  =  np.array([Epsilon_CI_Between_Effect_Lower, Epsilon_CI_Between_Effect_Upper])
    results["CI Epsilon Square Within Effect"]  =  np.array([Epsilon_CI_Within_Effect_Lower, Epsilon_CI_Within_Effect_Upper])
    results["CI Epsilon Square Interaction Effect"]  =  np.array([Epsilon_CI_Interaction_Effect_Lower, Epsilon_CI_Interaction_Effect_Upper])
    results["CI Partial Epsilon Square Between Effect"]  =  np.array([Partial_Epsilon_CI_Between_Effect_Lower, Partial_Epsilon_CI_Between_Effect_Upper])
    results["CI Partial Epsilon Square Within Effect"]  =  np.array([Partial_Epsilon_CI_Within_Effect_Lower, Partial_Epsilon_CI_Within_Effect_Upper])
    results["CI Partial Epsilon Square Interaction Effect"]  =  np.array([Partial_Epsilon_CI_Interaction_Effect_Lower, Partial_Epsilon_CI_Interaction_Effect_Upper])
    
    results["CI Omega Square Between Effect"]  =  np.array([Omega_CI_Between_Effect_Lower, Omega_CI_Between_Effect_Upper])
    results["CI Omega Square Within Effect"]  =  np.array([Omega_CI_Within_Effect_Lower, Omega_CI_Within_Effect_Upper])
    results["CI Omega Square Interaction Effect"]  =  np.array([Omega_CI_Interaction_Effect_Lower, Omega_CI_Interaction_Effect_Upper])
    
    results["CI Partial Omega Square Between Effect"]  =  np.array([Partial_Omega_CI_Between_Effect_Lower, Partial_Omega_CI_Between_Effect_Upper])
    results["CI Partial Omega Square Within Effect"]  =  np.array([Partial_Omega_CI_Within_Effect_Lower, Partial_Omega_CI_Within_Effect_Upper])
    results["CI Partial Omega Square Interaction Effect"]  =  np.array([Partial_Omega_CI_Interaction_Effect_Lower, Partial_Omega_CI_Interaction_Effect_Upper])
    
    results["CI Generalized Omega Square Between Effect"]  =  np.array([Generalized_Omega_CI_Between_Effect_Lower, Generalized_Omega_CI_Between_Effect_Upper])
    results["CI Generalized Omega Square Within Effect"]  =  np.array([Generalized_Omega_CI_Within_Effect_Lower, Generalized_Omega_CI_Within_Effect_Upper])
    results["CI Generalized Omega Square Interaction Effect"]  =  np.array([Generalized_Omega_CI_Interaction_Effect_Lower, Generalized_Omega_CI_Interaction_Effect_Upper])
    
    # Contrasts 
    results["Within Variable Contrasts Comparison"] = Within_Contrasts_Comparison
    results["Within Variable Mean Difference"] = Within_Mean_Difference
    results["Within Variable Standard Error"] = Within_Contrasts_Standard_Error
    results["Within Variable DF Denominator"] = Within_Contrasts_Degrees_of_freedom_Denominator
    results["Within Variable DF Numerator"] = Within_Contrasts_Degrees_of_freedom_Numerator
    results["Within Variable t Values"] = Within_Contrasts_t_values
    results["Within Variable F Values"] = Within_Contrasts_F_values
    results["Within Variable MS Error"] = Within_Contrast_MSerror
    results["Within Variable SS Error"] = Within_Contrasts_SSerror
    results["Within Variable Ms Contrast"] = Within_Contrasts_Ms_Contrast
    results["Within Variable SS Contrast"] = Within_Contrasts_SS_Contrast
    results["Between Variable Contrasts Comparison"] = Between_Contrasts_Comparison
    results["Between Variable Mean Difference"] = Between_Mean_Difference
    results["Between Variable Standard Error"] = Between_Contrasts_Standard_Error
    results["Between Variable DF Denominator"] = Between_Contrasts_Degrees_of_freedom_Denominator
    results["Between Variable DF Numerator"] = Between_Contrasts_Degrees_of_freedom_Numerator
    results["Between Variable t Values"] = Between_Contrasts_t_values
    results["Between Variable F Values"] = Between_Contrasts_F_values
    results["Between Variable MS Error"] = Between_Contrast_MSerror
    results["Between Variable SS Error"] = Between_Contrasts_SSerror
    results["Between Variable Ms Contrast"] = Between_Contrasts_Ms_Contrast
    results["Between Variable SS Contrast"] = Between_Contrasts_SS_Contrast

    # Simple Effects Within in Between

    # General
    results["Group's Name"] = 1
    results["Group Sample Size"] = 2
    results["        "] = " "

    # Method 1 - The Multivarite Method
    
    results["F_Within_in_Between"] = F_Within_In_Between_multivarite_1
    results["F_Within_in_Between (as in Joint_Test)"] = F_Within_In_Between_multivarite_2
    results["Wilks1"] = Within_in_Between_Simple_Effect_Wilks_1
    results["Wilks2"] = Within_in_Between_Simple_Effect_Wilks_2
    results["         "] = " "

    results["Partial Eta Square MultiVariate 1 (Tau Square)"] = Partial_Eta_Square_within_by_between_Multivariate1
    results["Partial Eta Square MultiVariate 2 (Tau Square)"] = Partial_Eta_Square_within_by_between_Multivariate2
    results["Partial Omega Square MultiVariate 1"] = Partial_Omega_Sqaure_within_by_between_Multivariate1
    results["Partial Omega Square MultiVariate 2"] = Partial_Omega_Sqaure_within_by_between_Multivariate2
    results["Partial Epsilon Square MultiVariate 1 (Adjusted Tau Square)"] = Partial_Epsilon_Square_within_by_between_Multivariate1
    results["Partial Epsilon Square MultiVariate 2 (Adjusted Tau Square)"] = Partial_Epsilon_Square_within_by_between_Multivariate2
    results["Partial Cohens f square 1 "] = Partial_Cohens_f_Multivariate1
    results["Partial Cohens f square 2"] = Partial_Cohens_f_Multivariate2


    # Method 2 Univariate Method: 
    results["Effect Sum of Squares Univariate Method"] = SS_Effect_Within_By_Between.values
    results["Error Sum of Squares Univariate Method"] = SS_Error_Within_By_Between.values
    results["Df Effect Univariate Method"] = np.repeat(df_within, Number_Of_Between_Variable_Groups)
    results["Df Error Univariate Method"] = df_Int_Subjects_Within
    results["MS Effect Univariate Method"] = Ms_Effect_Within_By_Between.values
    results["MS Error Univariate Method"] = Ms_Error_Within_By_Between_Univariate
    results["F Values Univariate Method"] = F_Within_in_Between_Univariate.values
    
    results["Partial Eta Square Univariate"] = Partial_Eta_Square_within_by_between_Univariate.values
    results["Partial Omega Square Univariate *needs to be checked"] = Partial_Omega_Sqaure_within_by_between_Univariate.values # I need to check how to do this one !!! 
    results["Partial Epsilon Square Univariate"] = Partial_Epsilon_Square_within_by_between_Univariate.values
    results["Partial Cohens f Univariate"] = Partial_Cohens_f2_within_by_between_Univariate.values

    results["Eta Square Univariate Method"] = Eta_Square_within_by_between_Univariate.values
    results["Omega Square Univariate"] = Omega_Square_within_by_between_Univariate.values
    results["Epsilon Square Univariate"] = Epsilon_Square_within_by_between_Univariate.values
    results["Cohens f Univariate"] = Cohens_f_Square_within_by_between_Univariate.values
    results["          "] = " "

    # Method 3: Seperate Method (Consider Adding Generalized Omega Square)
    results["Effect Sum of Squares Seperate"] = SS_Effect_Within_By_Between.values
    results["Error Sum of Squares Seperate"] = SS_Error_Within_By_Between.values
    results["Df Effect Seperate "] = np.repeat(df_within, Number_Of_Between_Variable_Groups)
    results["Df Error Seperate "] = Degrees_Of_Freedom_Error_Simple_Seperate
    results["MS Effect Seperate "] = Ms_Effect_Within_By_Between.values
    results["MS Error Seperate "] = Ms_Error_Within_By_Between_Seperate.values
    results["F Values Seperate "] = F_Within_in_Between_Seperate.values
    results["Partial Eta Square Seperate"] = Partial_Eta_Square_within_by_between_seperate.values
    results["Partial Omega Square Seperate * Needs to be checkes"] = Partial_Omega_Sqaure_within_by_between_seperate.values # I need to check that
    results["Partial Epsilon Square Seperate"] = Partial_Epsilon_Square_within_by_between_seperate.values
    results["Partial Cohens f square Seperate"] = Partial_Cohens_f2_within_by_between_seperate.values
    results["Eta Square Seperate "] = Eta_Square_within_by_between_seperate.values
    results["Omega Square Seperate"] = Omega_Square_within_by_between_seperate.values
    results["Epsilon Square Seperate"] = Epsilon_Square_within_by_between_seperate.values
    results["Cohens f square Seperate"] = Cohens_f_Square_within_by_between_seperate.values

    results["Effect Sum of Squares Simple Effect Between Subject"] = SS_Effect_Between_in_Within.values
    results["Error Sum of Squares Simple Effect Between Subject"] = SS_Error_Between_in_Within.values
    results["MS Effect Simple Effect Between Subject"] = MS_Effect_Between_in_Within.values
    results["MS Error Simple Effect Between Subject"] = MS_Error_Between_in_Within.values
    results["F-Values"] = F_values_Between_in_Within.values
    results["p_values_Between_in_Within"] = p_values_Between_in_Within
    results["Eta Square Between in Within"] = Eta_Square_Between_in_Within.values
    results["Omega Square Between in Within"] = Omega_Square_Between_in_Within.values
    results["Epsilon Square Between in Within"] = Epsilon_Square_Between_in_Within.values
    results["Cohens_f Square Between in Within"] = Cohens_f_Between_in_within.values

    results["Partial Eta Square Between in Within"] = Partial_Eta_Square_Between_in_Within.values
    results["Partial Omega Square Between in Within"] = Partial_Omega_Square_Between_in_Within.values
    results["Partial Epsilon Square Between in Within"] = Partial_Epsilon_Square_Between_in_Within.values
    results["Partial Cohens f Square Between in Within"] = Partial_Cohens_f.values

    ### DO NOT FORGET TO ADD PVALUES FOR ALL TESTS

    result_str = "\n".join([f"{key}: {value}" for key, value in results.items()])
    
    table = Table(title="Anova Table")
    rows = [
            ["John", "Doe", "45",""],
            ["Jane", "Doe", "32"],
            ["Mary", "Smith", "25"],
        ]
    columns = ["Effect", "SSeffect", "SSerror", "df", "MSeffect", "MSerror"]

    for column in columns:
        table.add_column(column)

    for row in rows:
        table.add_row(*row, style='red')

    console = Console()
    table_1 = console.print(table)
        
    return table_1



# Example usage with within=None:
result = Two_Way_Mixed(data2, dependent_variable='Target.RT', subject_variable='Subject', within=['Type'], between=['Group'])

from rich.console import Console
from rich.table import Table







R[write to console]: Converting to factor: Group

R[write to console]: Contrasts set to contr.sum for the following variables: Group

C:\Users\EldadKeha\AppData\Local\Temp\ipykernel_23684\1475565950.py:370: RuntimeWarning: invalid value encountered in sqrt
  Cohens_f = np.sqrt( (df_between / Number_Of_Measures) * (F_Statistic_Between_Effect - 1))


Effect Sum of Squares Deviations: [  89.60555556 2165.51111111 1098.84444444]
Error Sum of Squares Deviations: [2087.19444444 4640.55555556 4640.55555556]
Effect Degrees of Freedom: [1. 2. 2.]
Error Degrees of Freedom: [ 8. 16. 16.]
Effect Mean Squares Deviations: [  89.60555556 1082.75555556  549.42222222]
Error Mean Squares Deviations: [260.89930556 290.03472222 290.03472222]
SStotal: 9621.466666666667
F-Value: [0.34344881 3.73319286 1.89433258]
p-values: [0.57400315 0.04670789 0.18265344]
Eta Square                  : [0.00931309 0.22507079 0.11420758]
Epsilon Square              : [-0.01780329  0.1647817   0.05391849]
Omega Square                : [-0.01733327  0.16043139  0.05249502]
Generalized Eta Square      : [0.01409533 0.25678962 0.14917082]
Generalized Omega Square    : [-0.02288744  0.15329222  0.05924004]
Partial Eta Square          : [0.04116389 0.31817366 0.19145633]
Partial Epsilon Square      : [-0.07869062  0.23294536  0.09038837]
Partial Omega Square        : [-0.07